# Analysis

1. Generate reliability score.

2. K means based on reliability score.

3. Fit models to clusters.

In [ ]:
import sqlite3
import pprint
from collections import defaultdict

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

import pmdarima as pm
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from prophet import Prophet

In [ ]:
conn = sqlite3.connect("choice.db")

query = "SELECT * FROM prediction"

df = pd.read_sql_query(query, conn)

conn.close()

In [ ]:
df = df.rename(columns={
    'posting_date': 'date',
})

df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month

In [ ]:
df_month = df.groupby(['year', 'month'])['gross_weight'].sum().reset_index()
df_month['date'] = pd.to_datetime(df_month[['year', 'month']].assign(day=1))

Visualize total monthly donations.

In [ ]:
plt.figure()
plt.title("Total montly donations")
plt.plot(df_month['date'], df_month['gross_weight'])
plt.yscale('log')
plt.show()

Significant outliers **don't** correspond clearly to any natural disaster or recession.

Instead, they can be attributed to a single donor with dozens of million pound donations. This presents a problem for time series analysis since those behaviors can't be explained through time series, e.g., seasonality, trends, etc.

In [ ]:
df.to_csv("prediction.csv", index=False)

In [ ]:
# contribution mean

df = pd.read_csv("prediction.csv")
print(df['gross_weight'].describe())

Grab outliers within each branch.

In [ ]:
df['gross_weight'].plot(kind='hist', bins=30, edgecolor='black', logy=True)

Generate reliability scores for donors.

In [ ]:
n_months = 262 # since 2003

df_encoded = pd.get_dummies(df, columns=['donor_id'], dtype=int)
df_encoded = df_encoded.drop(['date', 'gross_weight', 'branch_code', 'storage_code'], axis=1)
df_encoded = df_encoded.groupby(['year', 'month'], as_index=False).sum()

df_encoded = df_encoded.map(lambda x: 1 if x != 0 else 0)
df_encoded.to_csv("encoded.csv", index=False)

In [ ]:
donors = df['donor_id'].unique()

donor_dict = {}

for donor in donors:
    if str(donor) == 'nan': continue
    donor = 'donor_id_' + str(donor)
    donor_dict[donor] = df_encoded[donor].sum()

for donor in donor_dict.keys():
    donor_dict[donor] /= n_months

print(donor_dict)

df_reliability = pd.DataFrame.from_dict(donor_dict, orient='index', columns=['reliability_score'])
df_reliability.to_csv("reliability.csv", index=False)

In [ ]:
df = pd.read_csv("reliability.csv")

In [ ]:
df = pd.DataFrame(df['reliability_score'])

e_scaled = StandardScaler().fit_transform(df) # this is super important

inertia = []
k_range = range(1, 20)

for k in k_range:
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(e_scaled)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(k_range, inertia, marker='o')
plt.title("Elbow Method for Optimal k")
plt.xlabel("Clusters (k)")
plt.ylabel("Inertia")
plt.grid(True)
plt.show()

In [ ]:
df_reliability = df_reliability.reset_index(names='donor_id')

print(df_reliability.columns)

In [ ]:
km = KMeans(n_clusters=3)
km.fit(e_scaled)

df_reliability['group'] = km.labels_

print(km.labels_)
print(df_reliability)

In [ ]:
# clean donor_id

df_reliability['donor_id'] = df_reliability['donor_id'].str.replace('donor_id_', '', regex=False)
print(df_reliability)

In [ ]:
df_prediction = pd.read_csv("data/prediction.csv")

df_merge = pd.merge(df_prediction, df_reliability, on='donor_id')

df_merge.to_csv("final.csv", index=False)

In [ ]:
df = pd.read_csv("final.csv")

In [ ]:
rename_dict = {
    'DRY': ['DRY', 'FRESH'],
    'FROZEN': ['FROZEN'],
    'REFRIG': ['REF', 'REFRIG', 'Refrigerated']
}

rename_map = {old: new for new, olds in rename_dict.items() for old in olds}

df['storage_code'] = df['storage_code'].map(lambda x: rename_map.get(x, x))

df.to_csv("data/final.csv", index=False)

Group by food type and reliability cluster.

In [ ]:
df_storage_dict = {storage: g for storage, g in df.groupby('storage_code')}

for storage in df_storage_dict.keys():
    df_storage_dict[storage] = {group: g for group, g in df_storage_dict[storage].groupby('group')}

In [ ]:
for food in df_storage_dict.keys():
    df_group_dict = df_storage_dict[food]
    for group in df_group_dict.keys():
        print(food, group, df_group_dict[group]['gross_weight'].median())

In [ ]:
for food in df_storage_dict.keys():
    df_group_dict = df_storage_dict[food]
    for group in df_group_dict.keys():
        print(food, group, df_group_dict[group].describe)

In [ ]:
# clean outliers
for food in df_storage_dict.keys():
    df_group_dict = df_storage_dict[food]
    for group in df_group_dict.keys():
        cluster = df_group_dict[group]
        weight = cluster['gross_weight']
        q1 = weight.quantile(0.25)
        q3 = weight.quantile(0.75)
        iqr = q3 - q1

        lower = q1 - 0.8 * iqr
        upper = q3 + 2 * iqr

        median = cluster['gross_weight'].median()

        cluster['gross_weight'] = cluster['gross_weight'].apply(lambda x: median if x < lower or x > upper else x)
        cluster.to_csv(f"data/clusters/final_{food}_{group}.csv")

In [ ]:
palette = sns.color_palette("coolwarm", 5)
colors = [palette[label] for label in range(3)]

for food in df_storage_dict.keys():
    df_group_dict = df_storage_dict[food]
    for group in df_group_dict.keys():
            # also f"data/clusters/final_{food}_{group}.csv"
            cluster = df_group_dict[group]

            plt.scatter(cluster['date'], cluster['gross_weight'], s = 1, c = colors[group])

plt.title("Donation Clusters")
plt.xticks([])
plt.xlabel("Date")
plt.ylabel("Donation weight")
plt.show()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
model_dict = {
    "ARIMA": defaultdict(dict),
    "ETS": defaultdict(dict),
    "Prophet": defaultdict(dict)
}

for food in df_storage_dict.keys():
    print(f"Processing food type: {food}")
    df_group_dict = df_storage_dict[food]

    for group in df_group_dict.keys():
        cluster = f"{food}_{group}"
        original_cluster_df = df_group_dict[group]

        print(f"  Processing cluster: {cluster} with raw shape {original_cluster_df.shape}")

        cluster_df = original_cluster_df.loc[:, ['date', 'gross_weight']].copy()

        cluster_df['date'] = pd.to_datetime(cluster_df['date'])

        cluster_df['year'] = cluster_df['date'].dt.year
        cluster_df['month'] = cluster_df['date'].dt.month

        cluster_df_agg = cluster_df.groupby(['year', 'month'])['gross_weight'].sum().reset_index()

        cluster_df_agg['date'] = pd.to_datetime(dict(
            year=cluster_df_agg['year'],
            month=cluster_df_agg['month'],
            day=1
        ))

        cluster_df_agg.set_index('date', inplace=True)

        assembled_series = cluster_df_agg['gross_weight']

        # fill missing
        if not assembled_series.empty:
            full_range = pd.date_range(start=assembled_series.index.min(), end=assembled_series.index.max(), freq='MS')
        else:
            print(f"    Skipping cluster {cluster}: Aggregated series is empty.")
            continue

        y_values_reindexed = assembled_series.reindex(full_range)

        y_values_filled = y_values_reindexed.interpolate(method='linear')

        # split
        try:
            split_point = len(y_values_filled) - 1
            train = y_values_filled[:split_point]
            test = y_values_filled[split_point:]
        except Exception as e:
             print(f"    Error during train/test split for cluster {cluster}: {e}")
             continue
        
        # check/validate
        if not train.empty and not test.empty:
             print(f"    Train shape: {train.shape}, Test shape: {test.shape}")
        else:
            print(f"    Skipping cluster {cluster}: Insufficient data for training or testing after split.")
            continue

        # drop nan
        train_cleaned = train.dropna()
        test_cleaned = test.dropna()

        if train_cleaned.empty or test_cleaned.empty:
             print(f"    Skipping cluster {cluster}: Not enough data after dropping remaining NaNs.")
             continue
        else:
            print(f"    Cleaned Train shape: {train_cleaned.shape}, Cleaned Test shape: {test_cleaned.shape}")

        # models
        for model_name in model_dict.keys():
            print(f"    Processing {model_name} for {cluster}")

            model = None
            forecast = None

            # arima
            if model_name == "ARIMA":
                try:
                    model = pm.auto_arima(
                        train_cleaned,
                        start_p=1, start_q=1,
                        max_p=5, max_q=5,
                        seasonal=True,
                        m=12,
                        stepwise=True,
                        error_action='ignore',
                        suppress_warnings=True,
                        n_fits=50
                    )
                    forecast = model.predict(len(test_cleaned))

                except Exception as e:
                    print(f"      ARIMA fitting failed for {cluster}: {e}")
                    continue

            elif model_name == "ETS":
                try:
                    model = ExponentialSmoothing(
                        train_cleaned,
                        trend='add',
                        seasonal='add',
                        seasonal_periods=12
                    ).fit()
                    forecast = model.forecast(len(test_cleaned))

                except Exception as e:
                    print(f"      ETS fitting failed for {cluster}: {e}")
                    continue

            # prohpet
            elif model_name == "Prophet":
                try:
                    train_prophet_df = pd.DataFrame({'ds': train_cleaned.index, 'y': train_cleaned.values})

                    model = Prophet(
                        yearly_seasonality=True,
                        weekly_seasonality=False,
                        daily_seasonality=False
                    )
                    model.fit(train_prophet_df)

                    future = pd.DataFrame({'ds': test_cleaned.index})
                    forecast_df = model.predict(future)

                    fig = model.plot(forecast_df)

                    ax = fig.gca()
                    ax.set_xlabel("Date")
                    ax.set_ylabel("Cluster weight")

                    plt.title(cluster)  # Optional title
                    plt.show()

                    forecast = forecast_df.set_index('ds').reindex(test_cleaned.index)['yhat'].values
                except Exception as e:
                    print(f"      Prophet fitting failed for {cluster}: {e}")
                    continue

            if model is not None and forecast is not None:
                model_dict[model_name][cluster]["model"] = model
                model_dict[model_name][cluster]["forecast"] = forecast.tolist()

                model_dict[model_name][cluster]["stats"] = {
                    "MAE": mean_absolute_error(test_cleaned, forecast) if len(test_cleaned) == len(forecast) else np.nan,
                    "MAPE": mean_absolute_percentage_error(test_cleaned, forecast) if len(test_cleaned) == len(forecast) else np.nan,
                    "RMSE": np.sqrt(mean_squared_error(test_cleaned, forecast)) if len(test_cleaned) == len(forecast) else np.nan
                }
            else:
                print(f"    Did not get a valid model or forecast for {model_name} in {cluster}")


Print results.

In [ ]:
pprint.pprint(model_dict)